In [2]:
import time
import os
import IPython.display as ipd

import gpflow
import tensorflow as tf
import numpy as np
import pandas as pd

from GP.expectations import analyticExpectation, unscentedExpectation, gaussHermiteExpectation, montecarloExpectation

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

tf.logging.set_verbosity(tf.logging.FATAL)
tf.random.set_random_seed(42)
np.random.seed(42)

In [ ]:
save_or_load = 'rerun'
if save_or_load not in ['save','load','rerun','loadsave']:
    raise Exception(f'Invalid operation {save_or_load}')

In [ ]:
if save_or_load in ['save', 'rerun']:
    max_dim = 5

    inducing = np.random.rand(20,max_dim).astype(gpflow.settings.float_type)
    full_mu = np.random.rand(1,max_dim).astype(gpflow.settings.float_type)
    full_C = np.random.rand(1,max_dim).astype(gpflow.settings.float_type)**2

    gaussHermiteExpectation = gaussHermiteExpectation(2)

In [ ]:
filename = f'./points/utghtest.hdf'
if save_or_load in ['save', 'rerun']:
    os.makedirs('./points/', exist_ok=True)

    bar_total = ipd.ProgressBar(max_dim)
    ipd.display(ipd.HTML('<h4>Progress:</h4>'))
    bar_total.display()
    results = pd.DataFrame(columns=['unscented', 'gauss-hermite', 'monte-carlo'], index=range(1,max_dim+1), dtype=float)
    time_elapsed = pd.DataFrame(columns=['unscented', 'gauss-hermite', 'monte-carlo'], index=range(1,max_dim+1), dtype='<m8[ms]')
    
#     trials = 10

    for D in range(1,max_dim+1):
        MC = montecarloExpectation(2**D)
        sess = tf.Session()

        with sess.as_default():
            pX = gpflow.probability_distributions.DiagonalGaussian(full_mu[:,:D], full_C[:,:D])
            kern = gpflow.kernels.SquaredExponential(D, ARD=True)
            feat = gpflow.features.InducingPoints(inducing[:, :D])
            arg = [(kern, feat)]

            with gpflow.decors.params_as_tensors_for(kern, feat):
                analytic = analyticExpectation(pX, *arg).eval()

                t = time.perf_counter()
                unscented = unscentedExpectation(pX, *arg).eval()
                time_elapsed.at[D, 'unscented'] = np.timedelta64(int((time.perf_counter()-t)*1e3), 'ms')

                t = time.perf_counter()
                gaussHermite = gaussHermiteExpectation(pX, *arg).eval()
                time_elapsed.at[D, 'gauss-hermite'] = np.timedelta64(int((time.perf_counter()-t)*1e3), 'ms')
                
                t = time.perf_counter()
                monteCarlo = MC(pX, *arg).eval()
                time_elapsed.at[D, 'monte-carlo'] = np.timedelta64(int((time.perf_counter()-t)*1e3), 'ms')

        sess.close()
        results.at[D, 'unscented'] = np.linalg.norm(unscented - analytic)
        results.at[D, 'gauss-hermite'] = np.linalg.norm(gaussHermite - analytic)
        results.at[D, 'monte-carlo'] = np.linalg.norm(monteCarlo - analytic)
        bar_total.progress += 1

    time_elapsed = time_elapsed.astype('<m8[ms]')
    if save_or_load == 'save':
        results.to_hdf(filename, key='results')
        time_elapsed.to_hdf(filename, key='time_elapsed')
elif save_or_load in ['load', 'loadsave']:
    results = pd.read_hdf('./points/utghtest.hdf', key='results')
    time_elapsed = pd.read_hdf('./points/utghtest.hdf', key='time_elapsed')

ipd.display(ipd.HTML('<span>Done</span>'))

In [ ]:
figsize = np.array([2.5,1.5])*2

_, ax = plt.subplots(1,1,figsize=figsize)
ax.plot(results.index, results['unscented'],label='Unscented Transformation', linestyle='--')
ax.plot(results.index, results['gauss-hermite'], label='Gauss-Hermite')
ax.plot(results.index, results['monte-carlo'], label='Monte-Carlo', linestyle='-.')
ax.set_ylabel('L² distance from analytic solution')
ax.set_xlabel('Input Dimension')
# ax.set_xticks(results.index)
ax.legend()
if save_or_load in ['save', 'loadsave']:
    os.makedirs('./figs/utghtest/', exist_ok=True)
    plt.savefig('./figs/utghtest/accuracy.pdf', bbox_inches='tight')
plt.show(ax)

_, ax = plt.subplots(1,1,figsize=figsize)
ax.plot(time_elapsed.index, time_elapsed['unscented'],label='Unscented Transformation', linestyle='--')
ax.plot(time_elapsed.index, time_elapsed['gauss-hermite'], label='Gauss-Hermite')
ax.plot(time_elapsed.index, time_elapsed['monte-carlo'], label='Monte-Carlo', linestyle='-.')
# ax.set_yscale('log')
ax.set_ylabel('Time Elapsed (s)')
ax.set_xlabel('Input Dimension')
# ax.set_xticks(time_elapsed.index)
ax.legend()
if save_or_load in ['save', 'loadsave']:
    plt.savefig('./figs/utghtest/time_elapsed.pdf', bbox_inches='tight')
plt.show(ax)

_, ax = plt.subplots(1,1,figsize=figsize)
ax.plot(time_elapsed.index, 2*time_elapsed.index,label='Unscented Transformation', linestyle='--')
ax.plot(time_elapsed.index, 2**time_elapsed.index, label='Gauss-Hermite')
ax.plot(time_elapsed.index, 2**time_elapsed.index, label='Monte-Carlo', linestyle='-.')
ax.set_yscale('log')
ax.set_ylabel('Points used')
ax.set_xlabel('Input Dimension')
# ax.set_xticks(time_elapsed.index)
ax.legend()
if save_or_load in ['save', 'loadsave']:
    plt.savefig('./figs/utghtest/points_used.pdf', bbox_inches='tight')
plt.show(ax)